</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization

In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_groq_response, test_cases=test_cases) # setting manual test cases
# node = EvolNode(mp, None, None, get_response=get_groq_response) # automatic generation of test cases 


node.evolve("i1", replace=True, max_attempts=3, num_runs=2)

print("Inspection on the generated code: \n", node.code)

import time 
time.sleep(3)

input_dict = test_inputs[0]
output_dict = node(input_dict) # Could still have error due to unsuccessful parsing of output
print("Output from the code: \n", output_dict)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Evaluating fitness:   0%|          | 0/4 [00:00<?, ?it/s]


KeyboardInterrupt: 

#### Benchmarking on Evolution Strategy
- We of course need a "mixture" of different evolution strategies, so a list of strategies instead of a single one to improve on the population ... (TBD)

In [4]:
# Population building phase ... 
from methods.llm import get_groq_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=3, meta_prompt=mp, get_response=get_groq_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Evaluating fitness: 100%|██████████| 4/4 [00:00<00:00, 6041.49it/s]


--- Replacing with new node
 - Attempt 1 failed. Fitness: 0.00. Error:  This node has no idea of what's going on
--- Compiled 0 out of 4 test cases
--- Passed 0 out of 4 test cases
Input: {'name': 'Dilireba'}, Output is missing or of wrong type, Expected: {'age': 32}
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}
Input: {'name': 'Dilireba'}, Output is missing or of wrong type, Expected: {'age': 32}
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}

Error Message:
--- Calling Prompt Function Error:
unexpected '{' in field name--- Calling Prompt Function Error:
unexpected '{' in field name--- Calling Prompt Function Error:
unexpected '{' in field name--- Calling Prompt Function Error:
unexpected '{' in field name


Evaluating fitness: 100%|██████████| 4/4 [00:02<00:00,  1.77it/s]


--- Replacing with new node
 - Attempt 2 failed. Fitness: 0.75. Error:  Function runs with success rate: 75.0%, runs correctly with rate: 75.0%
--- Compiled 3 out of 4 test cases
--- Passed 3 out of 4 test cases
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}

Error Message:
--- Calling Prompt Function Error:
Failed to parse LLM response: No JSON structure found in the provided text.


Evaluating fitness: 100%|██████████| 4/4 [00:07<00:00,  1.98s/it]


--- Replacing with new node
 - Attempt 3 failed. Fitness: 0.75. Error:  Function runs with success rate: 75.0%, runs correctly with rate: 75.0%
--- Compiled 3 out of 4 test cases
--- Passed 3 out of 4 test cases
Input: {'name': 'ChengXiao'}, Output is missing or of wrong type, Expected: {'age': 26}

Error Message:
--- Calling Prompt Function Error:
Failed to parse LLM response: No JSON structure found in the provided text.
Evolution failed after 3 attempts.
Based on the provided information, let me analyze the effectiveness of the current evolution strategy:

1. Fitness Improvement:
- Initial best fitness: 1.0
- Current best fitness after evolution: 0.75
- This actually shows a decrease in fitness, which suggests that the evolution strategy might not be improving the solution in terms of fitness metrics.

2. Implementation Improvements:
Despite the lower fitness score, there are several notable improvements in the implementation:

a) Added Functionality:
- Introduced the `search_google


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="1000" alt="Fourier reconstruction convergence">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>

</div>
<div align="center">
  <img src="img/Planning-node.png" width="400" alt="Planning Node Task Decomposition">
  <p><em> But how do we know what are the tasks suitable for our goal? Design of tasks topology is the fundation of planning, let's ask LLM for help on this, too! Evolution Graph autuomate planning by imagning topology of tasks which works best for your goal.</em></p>
</div>


In [1]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan,extract_python_code, extract_json_from_text
from methods.evolnode import PlanNode


# Step 0. Initialize MetaPlanning Prompt
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_groq_response)


plan_dict = plan._evolve_plan_dict() # differentiates retrieved existing nodes and new hypothetical nodes

In [9]:
# visualize_plan_dict(plan_dict)
# plan.referrable_function_dict
# print(plan.relevant_node_desc)

retrieved_dict = plan.referrable_function_dict


retrieved_dict.keys()

dict_keys(['search_google'])

In [34]:
node = plan.relevant_nodes[0]

# type(node)


In [39]:
list(retrieved_dict.keys())

def _update_plan_dict(plan, plan_dict):
    for node in plan.relevant_nodes:
        for sub_node in plan_dict["nodes"]:
            if node.meta_prompt.func_name == sub_node["name"]:
                for k in ["inputs", "input_types", "outputs", "output_types"]:
                    sub_node[k] = getattr(node.meta_prompt, k)
                for k in ["code", "reasoning", "fitness"]:
                    sub_node[k] = getattr(node, k)
    return plan_dict 


Input values into planned sub-node


In [40]:
plan_dict["nodes"]

[{'task': "Search for celebrity's Wikipedia page.",
  'name': 'search_google',
  'inputs': ['query'],
  'input_types': ['str'],
  'outputs': ['result'],
  'output_types': ['str'],
  'target': "Obtain celebrity's Wikipedia page URL.",
  'mode': 'PROMPT',
  'code': '\nimport http.client\nimport json\nimport os\nfrom typing import Dict, Any\n\ndef _search_google(query: str) -> Dict[str, Any]:\n    """\n    Use Serper API to search Google for information\n    \n    Args:\n        query (str): The search query\n    \n    Returns:\n        Dict[str, Any]: Parsed JSON response from the API\n    """\n    conn = http.client.HTTPSConnection("google.serper.dev")\n    payload = json.dumps({"q": query})\n    headers = {\n        \'X-API-KEY\': os.environ["SERPER_API_KEY"],\n        \'Content-Type\': \'application/json\'\n    }\n    \n    try:\n        conn.request("POST", "/search", payload, headers)\n        res = conn.getresponse()\n        data = res.read()\n        return json.loads(data.decode

In [29]:
# func_dict[k]
# k
# func_dict

retrieved_dict

{'search_google': '\nimport http.client\nimport json\nimport os\nfrom typing import Dict, Any\n\ndef _search_google(query: str) -> Dict[str, Any]:\n    """\n    Use Serper API to search Google for information\n    \n    Args:\n        query (str): The search query\n    \n    Returns:\n        Dict[str, Any]: Parsed JSON response from the API\n    """\n    conn = http.client.HTTPSConnection("google.serper.dev")\n    payload = json.dumps({"q": query})\n    headers = {\n        \'X-API-KEY\': os.environ["SERPER_API_KEY"],\n        \'Content-Type\': \'application/json\'\n    }\n    \n    try:\n        conn.request("POST", "/search", payload, headers)\n        res = conn.getresponse()\n        data = res.read()\n        return json.loads(data.decode("utf-8"))\n    except Exception as e:\n        print(f"Error occurred during API request: {str(e)}")\n        return {}\n    finally:\n        conn.close()\n        \ndef search_google(query: str) -> str: \n    """ \n    Input query, return sear

In [19]:
sub_node

{'task': "Search for celebrity's Wikipedia page.",
 'name': 'search_google',
 'inputs': ['query'],
 'input_types': ['str'],
 'outputs': ['result'],
 'output_types': ['str'],
 'target': "Obtain celebrity's Wikipedia page URL.",
 'mode': 'PROMPT'}

In [17]:
plan_dict["nodes"][0]

{'task': "Search for celebrity's Wikipedia page.",
 'name': 'search_google',
 'inputs': ['query'],
 'input_types': ['str'],
 'outputs': ['result'],
 'output_types': ['str'],
 'target': "Obtain celebrity's Wikipedia page URL.",
 'mode': 'PROMPT'}

In [1]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan,extract_python_code, extract_json_from_text
from methods.evolnode import PlanNode


# Step 0. Initialize MetaPlanning Prompt
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_groq_response)


# # Step 1: Generate Pseudo-Code for SubTask Decomposition
# prompt = mp._get_pseudo_code_prompt() # Pseudo-Code Prompt (Non-implemented functional)
# plan.query_nodes(ignore_self=False, self_func_name=None)

# response = get_groq_response(prompt) # Use Strong LLM to build up pseudo-code
# code = extract_python_code(response) # Extract Python Code from response 

# # Step 2: Generate Planning DAG: Multiple Nodes 
# graph_prompt = mp._get_plan_graph_prompt(code) 
# plan_response = get_groq_response(graph_prompt)
# plan_dict = extract_json_from_text(plan_response)

# visualize_plan_dict(plan_dict)

# Step 3: Spawn Multiple Sub-Nodes and Evolve them
# for sub_node_dict in plan_dict["nodes"]:
#     meta_prompt = MetaPrompt.from_dict(sub_node_dict)
#     # TBD: query nodes for dynamic planning
#     node = EvolNode(meta_prompt, None, None, get_response=get_claude_response) # automatically generate test cases
#     node.get_response = get_groq_response # for generation we use groq to speed it up
#     node.evolve("i1", replace=True, max_attempts=3, num_runs=2)
#     node.save() # Just slot into library for now ... || TBD: only save nodes which cross certain quality threshold

# Step 4: Slot Sub Node into context for parent node re-write 


In [3]:
mp.task

'Get the age of celebrity.'